In [1]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from collections import Counter

In [2]:
def scrape(html):
    soup = BeautifulSoup(html, "html.parser")
    json_data = json.loads(soup.find_all("script", type="application/ld+json")[-1].text)
    title = json_data.get('headline', soup.find('h1', class_='content--title').text)
    date = json_data.get('datePublished', re.search(r'datetime:.*?(\d{4}-\d{2}-\d{2}T\d{2}:\d{2})', str(html)).group(1))
    date_m = json_data.get('dateModified', '')
    genre = json_data.get('genre', [])
    if genre == []:
        genre = [k for k in soup.find('meta', attrs={'name':'keywords'}).get('content').split(',') if k not in ['NHK','ニュース', 'NHK NEWS WEB']]
    keywords = json_data.get('keywords', [])
    article = soup.find('div', id="news_textbody").text
    url_normal = 'https:' + soup.find('meta', property="og:url").get('content').rsplit('https:')[-1]
    if soup.find_all('div', id="news_textmore") != []:
        for textmore in soup.find_all('div', id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all('div', class_="news_add") != []:
        for newsadd in soup.find_all('div', class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url_normal.split('/')[-1].split('.html')[0],
        'title':title,
        'article':article.strip(),
        'genre':genre,
        'keywords':keywords,
        'url':url_normal,
        'datePublished':date,
        'dateModified':date_m
    }

def remove_rt(text):
    return re.sub('<rt>.+?</rt>', '', text)

def tag(text):
    text = re.sub(r'<span class="colorC">(.+?)</span>', r"{org}\1{/org}", text)
    text = re.sub(r'<span class="colorL">(.+?)</span>', r"{plc}\1{/plc}", text)
    text = re.sub(r'<span class="colorN">(.+?)</span>', r"{per}\1{/per}", text)
    return text

def retag(text):
    text = re.sub(r'{org}(.+?){/org}', r"<org>\1</org>", text)
    text = re.sub(r'{plc}(.+?){/plc}', r"<plc>\1</plc>", text)
    text = re.sub(r'{per}(.+?){/per}', r"<per>\1</per>", text)
    return text

def remove_a(text):
    text = re.sub(r'</?a.*?>', '', text)
    text = re.sub(r'<span class="under">(\w+)</span>', r'\1', text)
    return text

# for old web easy
def scrape_easy_one(html):
    soup = BeautifulSoup(html, "html.parser")
    url_normal = soup.find('div', id="regularnews").a.get('href').split('/http://')[-1]
    url_normal = 'http://' + url_normal
    date = soup.find('p', id="newsDate").text[1:-1]
    #title_easy = soup.find('h1', class_="article-main__title")
    #title_easy_ruby = ''.join([str(t) for t in title_easy.contents]).strip()
    url_easy = soup.find('meta', attrs={'name':'shorturl'}).get('content')
    title_easy = soup.find('div', id='newstitle').h2
    title_easy_ruby = ''.join([str(t) for t in title_easy.contents]).strip()
    title_easy = BeautifulSoup(remove_rt(str(title_easy)), "html.parser").text.strip()
    article_easy = soup.find('div', id="newsarticle")
    article_easy = BeautifulSoup(tag(remove_rt(str(article_easy))), "html.parser").text.strip()
    article_easy_ruby = soup.find('div', id="newsarticle").find_all('p')
    article_easy_ruby = '\n'.join([''.join([remove_a(str(l)) for l in p.contents]) for p in article_easy_ruby if p != []]).strip()
    
    return {
        'id':url_easy.split('/')[-1].split('.html')[0],
        'title_easy':title_easy,
        'title_easy_ruby':title_easy_ruby,
        'article_easy':retag(article_easy),
        'article_easy_ruby':article_easy_ruby,
        'url_easy':url_easy,
        'url_normal':url_normal,
        'date_easy':date
    }

def get_link(start=0):
    notyet = []
    n_list = pd.read_json('nhkweb.json', encoding='utf-8')['url'].tolist()
    df_e = pd.read_json('nhkwebeasy.json', encoding='utf-8') 
    for i in df_e['url_normal'][start:]:
        if i not in n_list:
            notyet.append(i)
    return notyet

def get_link_nogenre(start=0):
    notyet = []
    with open('nhkweb.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    for dic in data:
        if dic['genre'] == []:
            notyet.append(dic['url'])
    return notyet  

def js(dic):
    with open('nhkweb.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open('nhkweb.json', 'w', encoding='utf-8') as f:
        if dic['id'] not in [x['id'] for x in data]:
            data.append(dic)
        else:
            for i, d in enumerate(data):
                if dic['id'] == d['id']:
                    data[i] = dic
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)
        
def js_e(dic):
    with open('nhkwebeasy.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open('nhkwebeasy.json', 'w', encoding='utf-8') as f:
        if dic['id'] not in [x['id'] for x in data]:
            data.append(dic)
        else:
            for i, d in enumerate(data):
                if dic['id'] == d['id']:
                    data[i] = dic
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)

In [4]:
htmls = ['https://www3.nhk.or.jp/news/html/20191117/k10012180511000.html']

In [28]:
htmls = get_link(0); htmls

['https://www3.nhk.or.jp/news/html/20190120/k10011784611000.html',
 'https://www3.nhk.or.jp/news/html/20191010/k10012120321000.html',
 'https://www3.nhk.or.jp/news/html/20191017/k10012135411000.html',
 'https://www3.nhk.or.jp/news/html/20200204/k10012271251000.html',
 'https://www3.nhk.or.jp/news/html/20200204/k10012271311000.html']

In [29]:
htmls = get_link_nogenre(); htmls[:50]

[]

# scrape

In [18]:
driver = webdriver.Chrome()

In [16]:
for nhkurl in tqdm.tqdm(htmls[:]):
    driver.get(f'https://web.archive.org/web/2019*/{nhkurl}')
    time.sleep(10)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'(between|1 time).*?<a href="(.+?)">', html)
    if not snap:
        print(nhkurl)
        continue
    driver.get('https://web.archive.org' + snap.group(2))
    time.sleep(30)
    html = driver.page_source.encode('utf-8')
    dic = scrape(html)
    js(dic)

 14%|█▍        | 1/7 [01:22<08:16, 82.81s/it]

KeyboardInterrupt: 

In [94]:
html = driver.page_source.encode('utf-8')
soup = BeautifulSoup(html, "html.parser")
dic = scrape(html)
print(dic)
js(dic)

{'id': 'k10011846961000', 'title': '買い物弱者など解決へ ロボットがコンビニの商品配達する実験', 'article': '高齢化に伴う「買い物弱者」などの解決につなげようと、自動で動くロボットがコンビニの商品を配達する実験が神奈川県の大学で行われました。\nこれは大手コンビニチェーンの「ローソン」と自動運転技術を開発するベンチャー企業の「ＺＭＰ」などが行いました。実験は大学内で行われ、利用者がスマートフォンの専用のアプリを使って商品を注文すると、高さ、幅などが１メートルに満たない箱型のロボットが、仮設の店舗から商品を届けます。センサーやカメラが周りの状況を認識しているということで、人が近づくと自動的に停止するほか、人とすれ違った際には音声であいさつもしてくれます。両社では、こうしたロボットで高齢化に伴う買い物弱者やドライバーの人手不足といった課題の解決につなげたい考えです。また公道を移動できるよう国に対して法整備を働きかけていくことにしています。ローソンの牧野国嗣オープン・イノベーションセンター長は「時間がなかったり、体が不自由で店に行けなかったりするなど、さまざまなニーズに応えるため、実用化を進めていきたい」と話していました。', 'genre': ['ビジネス', '社会'], 'keywords': [], 'url': 'https://www3.nhk.or.jp/news/html/20190313/k10011846961000.html', 'datePublished': '2019-03-13T17:35:18+09:00', 'dateModified': '2019-03-27T14:00:23+09:00'}


In [91]:
html = str(driver.page_source.encode('utf-8'))
re.search(r'"genre":(.+?),', html)

# clean category & keyword

In [17]:
with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)
    print(len(data))
with open('nhkwebeasy.json','r', encoding='utf-8') as f:
    data = json.load(f)
    print(len(data))

5147
1685


In [18]:
# check category

with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)
print('articles: ', len(data))
genre = Counter()
for dic in data:
    for g in dic['genre']:
        genre[g] += 1
genre.most_common()

articles:  5147


[('社会', 1639),
 ('国際', 1322),
 ('ビジネス', 823),
 ('科学・文化', 729),
 ('スポーツ', 702),
 ('政治', 615),
 ('暮らし', 410),
 ('地域', 319),
 ('気象・災害', 271)]

In [24]:
# genre <> keywords

with open('nhkweb.json','r', encoding='utf-8') as f:
    data = json.load(f)

category = ['社会', '国際', 'ビジネス', 'スポーツ', '政治', '科学・文化', '暮らし', '地域', '気象・災害']
for i, dic in enumerate(data):
    newgenre = []
    newkey = []
    for j in dic['genre']:
        if j in category:
            newgenre.append(j)
        elif j == "災害" or j == "気象":
            newgenre.append('気象・災害')
        else:
            newkey.append(j)
    for j in dic['keywords']:
        if j in category:
            newgenre.append(j)
        elif j == "災害" or j == "気象":
            newgenre.append('気象・災害')
        else:
            newkey.append(j)
    data[i]['genre'] = list(set(newgenre))
    data[i]['keywords'] = list(set(newkey))

with open('nhkweb.json','w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)
    

In [3]:
normal = pd.read_json('nhkweb.json')
normal.id.value_counts()

k10012255531000    1
k10012263311000    1
k10012263971000    1
k10012228721000    1
k10012251101000    1
                  ..
k10011795521000    1
k10012234731000    1
k10012253721000    1
k10012084571000    1
k10012239591000    1
Name: id, Length: 5147, dtype: int64

# web easy archive

In [9]:
with open('tobescraped.txt') as f:
    lst = f.read().split()
with open('nolink.txt') as f:
    nolink = f.read().split()
with open('nhkwebeasy.json') as f:
    urls = [x['url_easy'] for x in json.load(f)]
htmls = sorted(set(lst) - set(urls) - set(nolink))
print(len(htmls))
htmls[30:40]

634


['http://www3.nhk.or.jp/news/easy/k10010443301000/k10010443301000.html',
 'http://www3.nhk.or.jp/news/easy/k10010443481000/k10010443481000.html',
 'http://www3.nhk.or.jp/news/easy/k10010443511000/k10010443511000.html',
 'http://www3.nhk.or.jp/news/easy/k10010443941000/k10010443941000.html',
 'http://www3.nhk.or.jp/news/easy/k10010443971000/k10010443971000.html',
 'http://www3.nhk.or.jp/news/easy/k10010444561000/k10010444561000.html',
 'http://www3.nhk.or.jp/news/easy/k10010444601000/k10010444601000.html',
 'http://www3.nhk.or.jp/news/easy/k10010445021000/k10010445021000.html',
 'http://www3.nhk.or.jp/news/easy/k10010445391000/k10010445391000.html',
 'http://www3.nhk.or.jp/news/easy/k10010445501000/k10010445501000.html']

In [4]:
driver = webdriver.Chrome()

In [10]:
for nhkurl in tqdm.tqdm(htmls[20:200]):
    driver.get(f'https://web.archive.org/web/2016*/{nhkurl}')
    time.sleep(10)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'(between|1 time).*?<a href="(.+?)">', html)
    if not snap:
        print(nhkurl)
        continue
    driver.get('https://web.archive.org' + snap.group(2))
    time.sleep(20)
    html = driver.page_source.encode('utf-8')
    dic = scrape_easy_one(html)
    js_e(dic)



  0%|          | 0/180 [00:00<?, ?it/s]

  1%|          | 1/180 [01:01<3:04:25, 61.82s/it]

  1%|          | 2/180 [01:57<2:58:03, 60.02s/it]

  2%|▏         | 3/180 [02:54<2:53:50, 58.93s/it]

  2%|▏         | 4/180 [03:55<2:54:53, 59.62s/it]

  3%|▎         | 5/180 [04:54<2:53:45, 59.57s/it]

  3%|▎         | 6/180 [06:19<3:14:18, 67.00s/it]

  4%|▍         | 7/180 [07:41<3:26:38, 71.67s/it]

  4%|▍         | 8/180 [08:37<3:12:10, 67.04s/it]

  5%|▌         | 9/180 [09:29<2:58:05, 62.49s/it]

  6%|▌         | 10/180 [10:21<2:47:53, 59.26s/it]

  6%|▌         | 11/180 [11:07<2:35:31, 55.21s/it]

  7%|▋         | 12/180 [11:51<2:25:01, 51.79s/it]

  7%|▋         | 13/180 [12:32<2:15:27, 48.67s/it]

  8%|▊         | 14/180 [13:20<2:14:19, 48.55s/it]

  8%|▊         | 15/180 [14:16<2:19:15, 50.64s/it]

  9%|▉         | 16/180 [15:05<2:17:39, 50.36s/it]

  9%|▉         | 17/180 [16:01<2:21:27, 52.07s/it]

 10%|█         | 18/180 [16:49<2:17:00, 50.74s/it]

 11%|█         | 19/180 [17:41

In [7]:
for nhkurl in tqdm.tqdm(htmls[40:150]):
    driver.get(f'https://web.archive.org/web/2016*/{nhkurl}')
    time.sleep(10)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'(between|1 time).*?<a href="(.+?)">', html)
    if not snap:
        print(nhkurl)
        continue
    driver.get('https://web.archive.org' + snap.group(2))
    time.sleep(20)
    html = driver.page_source.encode('utf-8')
    dic = scrape_easy_one(html)
    js_e(dic)

 54%|█████▎    | 59/110 [56:01<1:04:36, 76.01s/it]

AttributeError: 'NoneType' object has no attribute 'a'

In [6]:
html = driver.page_source.encode('utf-8')
dic = scrape_easy_one(html)
js_e(dic)